In [6]:
"""
多线程回顾
"""
from threading import Thread
def spider():
    print("也许时间是一种解药")

t_list=[]
for i in range(5):
    t=Thread(target=spider)
    t_list.append(t)
    t.start()

for t in t_list:
    t.join()  #阻塞






也许时间是一种解药
也许时间是一种解药
也许时间是一种解药
也许时间是一种解药
也许时间是一种解药


In [14]:
#线程锁，因为有时候共享变量，所以要进行上锁  
from threading import Thread,Lock
n=5000
def f1():
    global n
    for i in  range(1000000):
        Lock().acquire()
        n+=1
        Lock().release()
def f2():
    global n
    for i in  range(1000000):
        Lock().acquire()
        n-=1
        Lock().release()

t1=Thread(target=f1)
t1.start()
t2=Thread(target=f2)
t2.start()

t1.join()
t2.join()

print(n)  #不加锁，每一次都不一样

5000


Exception in thread Thread-48:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-e1cd40e12ccf>", line 9, in f1
    Lock().release()
RuntimeError: release unlocked lock

Exception in thread Thread-49:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-e1cd40e12ccf>", line 15, in f2
    Lock().release()
RuntimeError: release unlocked lock



In [18]:
"""
小米商城  多线程的爬取
"""
import requests
import time
import random
from fake_useragent import UserAgent
import json
from threading import Thread, Lock
from queue import Queue

class Xiaomi:
    def __init__(self):
        self.url='https://app.mi.com/categotyAllListApi?page={}&categoryId=2&pageSize=30'
        #创建队列
        self.q=Queue()
        self.lock=Lock()
        
    def url_in(self):
        #将URL入队列
        for page in range(0,3):
            url=self.url.format(page)
            self.q.put(url)
    
    def parse_html(self):
        """线程的时间函数，获取URL，处理数据"""
        while True:
            self.lock.acquire()#上锁
            if not self.q.empty():
                url=self.q.get()  #这里有点小问题，可能存在一个进程判断完，还没来得及取，下一个进程又开始判断了，容易造成阻塞
                self.lock.release()#解锁
                headers={"User-Agent":UserAgent().random}
                html=requests.get(url=url,headers=headers).json()
                item={}
                list=[] #试试json保存
                for app_dict in html["data"]:
                    item["name"]=app_dict["displayName"]
                    item["type"]=app_dict["level1CategoryName"]
                    item["link"]="https://app.mi.com/details?id="+app_dict["packageName"]
                    print(item)
            else:
                self.lock.release()#解锁,防止走这个没释放，当队列为空的时候，已经上锁了，但是没有释放就尴尬了
                break

    def run(self):
        #先让URL入队列
        self.url_in()
        #创建多线程运行
        t_list=[]
        for i in range(2):
            t=Thread(target=self.parse_html)
            t_list.append(t)
            t.start()
        
        #统一阻塞等待回收线程：
        for t in t_list:
            t.join()


if __name__ == "__main__":
    start_time=time.time()
    spider=Xiaomi()  #实例化对象的时候执行__init__
    spider.run()
    end_time=time.time()
    print("time:%.2f"%(end_time-start_time))




{'name': '闪令', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.ifitU.Vmuse'}
{'name': '回家儿', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.gtgy.countryn'}
{'name': '红线相亲', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.youyue.hx'}
{'name': '以陌语音-听歌听故事聊天聊CP', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.dadu.yimo'}
{'name': '铛铛', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.aladdin.dangdang'}
{'name': '红线交友', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.youyue.hxjy'}
{'name': '扩列漂流瓶', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=dream.com.piao'}
{'name': '论否', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=tech.nani.www.nani'}
{'name': '牵手', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.tantan.x'}
{'name': '宇宙奶茶馆-原甜味陪伴', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.sj.sweet'}
{'name': '希壤', 'type': '聊天社交', 'link': 'https://app.mi.com/details?id=com.baidu.xirang'}
{'